In [ ]:
!pip install -U liblinear-official

In [ ]:
from liblinear.liblinearutil import *

import requests
import numpy as np
import random
import time

# Data Preprocessing

In [ ]:
def transform_3(matrix):
  dim = len(matrix[1])
  new_mat = []
  
  for i, row_vec in enumerate(matrix):
    first_orders = row_vec[1:]
    row = row_vec[:]
    for j in range(0,dim-1):
      for k in range(j,dim-1):
        row.append( first_orders[j]*first_orders[k] )
        for l in range(k,dim-1):
          row.append( first_orders[j]*first_orders[k]*first_orders[l] )

    new_mat.append( row )

  return new_mat

In [ ]:
# Get Data
url = 'http://www.csie.ntu.edu.tw/~htlin/course/ml21fall/hw4/hw4_train.dat'
content = requests.get(url).content
content = content.decode('utf-8')

inXs = []
inYs = []
content = content.strip().split('\n')
for c in content:
    c = c.replace('\t',' ')
    c = [float(x) for x in c.split(' ')]
    inXs.append( [1]+c[:-1] )
    # inYs.append( [c[-1]] )
    inYs.append( c[-1] )


url = 'http://www.csie.ntu.edu.tw/~htlin/course/ml21fall/hw4/hw4_test.dat'
content = requests.get(url).content
content = content.decode('utf-8')

outXs = []
outYs = []
content = content.strip().split('\n')
for c in content:
    c = c.replace('\t',' ')
    c = [float(x) for x in c.split(' ')]
    outXs.append( [1]+c[:-1] )
    # outYs.append( [c[-1]] )
    outYs.append( c[-1] )


In [ ]:
inXs = transform_3(inXs)
outXs = transform_3(outXs)

In [ ]:
print(len(inYs), len(inXs))

200 200


In [ ]:
print( len(inXs[5]) )

84


# Problem 12

In [ ]:
prob = problem(inYs, inXs)

for lambda_ in [-4,-2,0,2,4]:
  print(lambda_, end=' ')
  param = parameter(f'-s 0 -c {1/(2*10**lambda_)} -e 0.000001')
  m = train(prob, param)
  p_label, p_acc, p_val = predict(outYs, outXs, m)
  # print(p_label)
  # ACC, MSE, SCC = evaluations(outYs, p_label)

-4 Accuracy = 87.625% (701/800) (classification)
-2 Accuracy = 87.75% (702/800) (classification)
0 Accuracy = 92% (736/800) (classification)
2 Accuracy = 92.75% (742/800) (classification)
4 Accuracy = 83.5% (668/800) (classification)


> Best param: lambda = -2

# Problem 13

In [ ]:
prob = problem(inYs, inXs)

for lambda_ in [-4,-2,0,2,4]:
  print(lambda_, end=' ')
  param = parameter(f'-s 0 -c {1/(2*10**lambda_)} -e 0.000001')
  m = train(prob, param)
  p_label, p_acc, p_val = predict(inYs, inXs, m, '-b 1')
  ACC, MSE, SCC = evaluations(inYs, p_label)

-4 Accuracy = 100% (200/200) (classification)
-2 Accuracy = 100% (200/200) (classification)
0 Accuracy = 97.5% (195/200) (classification)
2 Accuracy = 95% (190/200) (classification)
4 Accuracy = 82% (164/200) (classification)


> best param: -2


# Problem 14


In [ ]:
inX_train = inXs[:120]
inY_train = inYs[:120]
inX_val = inXs[120:]
inY_val = inYs[120:]

In [ ]:
prob = problem(inY_train, inX_train)

for lambda_ in [-4,-2,0,2,4]:
  print(lambda_, end=' ')
  param = parameter(f'-s 0 -c {1/(2*10**lambda_)} -e 0.000001')
  m = train(prob, param)

  p_label, p_acc, p_val = predict(inY_val, inX_val, m, '-b 1')

-4 Accuracy = 77.5% (62/80) (classification)
-2 Accuracy = 80% (64/80) (classification)
0 Accuracy = 88.75% (71/80) (classification)
2 Accuracy = 93.75% (75/80) (classification)
4 Accuracy = 85% (68/80) (classification)


> best param: 2

In [ ]:
param = parameter(f'-s 0 -c {1/(2*10**(2))} -e 0.000001')
m = train(prob, param) 

In [ ]:
p_label, p_acc, p_val = predict(outYs, outXs, m)
ACC, MSE, SCC = evaluations(outYs, p_label)

Accuracy = 92.5% (740/800) (classification)


# Problem 15

In [ ]:
prob = problem(inYs, inXs)

param = parameter(f'-s 0 -c {1/(2*10**(2))} -e 0.000001')
m = train(prob, param)

p_label, p_acc, p_val = predict(outYs, outXs, m)
ACC, MSE, SCC = evaluations(outYs, p_label)

Accuracy = 92.75% (742/800) (classification)


# Problem 16

In [ ]:
start = 0
ERRS = [0]*5

for round, lambda_ in enumerate([-4,-2,0,2,4]):
  start = 0
  for i in range(5):
    Xval = inXs[start:start+40]
    Yval = inYs[start:start+40]
    if start >= 0 and start+40<len(inXs):
      Xtrain = inXs[:start] + inXs[start+40:]
      Ytrain = inYs[:start] + inYs[start+40:]

    elif start+40>=len(inXs):
      Xtrain = inXs[:start]
      Ytrain = inYs[:start]

    # print(len(Xval), len(Xtrain))
    # print('tr',start, start+40)
    # print(Xtrain)
    prob = problem(Ytrain, Xtrain)
    param = parameter(f'-s 0 -c {1/(2*10**lambda_)} -e 0.000001')
    m = train(prob, param)
    p_label, p_acc, p_val = predict(Yval, Xval, m, '-b 1')
    # ACC, MSE, SCC = evaluations(Yval, p_label)
    print(p_acc, p_val)
    err = (1 - p_acc[0]/100)*40
    ERRS[round] += err/(5*40)

    start += 40
  

In [ ]:
print(-4,-2,0,2,4)
print(ERRS)

-4 -2 0 2 4
[0.19000000000000003, 0.17, 0.11499999999999999, 0.08000000000000002, 0.21]


1.   argmin = 2
2.   E_{cv} = 0.08

> 縮排段落



In [1]:
%%shell
jupyter nbconvert --to html /content/HW4.ipynb

[NbConvertApp] Converting notebook /content/HW4.ipynb to html
[NbConvertApp] Writing 307728 bytes to /content/HW4.html
